In [ ]:
import numpy as np
import os

In [ ]:
def load_coeffs(npz_file_path):
    """
    从npz文件加载小波系数。
    
    参数:
    npz_file_path: npz文件的路径。
    
    返回:
    cA, cH, cV, cD: 近似系数、水平细节系数、垂直细节系数、对角细节系数。
    """
    data = np.load(npz_file_path)  # 加载npz文件
    cA = data['cA']  # 近似系数
    cH = data['cH']  # 水平细节系数
    cV = data['cV']  # 垂直细节系数
    cD = data['cD']  # 对角细节系数
    return cA, cH, cV, cD

In [ ]:
def reduce_dimensions_separately(coeffs, target_dims):
    """
    分别对不同类型的小波系数进行降维。
    
    参数:
    coeffs: 包含cA, cH, cV, cD的列表。
    target_dims: 字典，指定每种系数的目标维度，例如{'cA': 600, 'cH': 133, 'cV': 133, 'cD': 134}。
    
    返回:
    reduced_coeffs: 降维后的系数列表。
    """
    reduced_coeffs = []  # 初始化降维后的系数列表
    for i, coeff in enumerate(coeffs):
        coeff_type = ['cA', 'cH', 'cV', 'cD'][i]  # 当前系数的类型
        target_dim = target_dims[coeff_type]  # 获取当前系数类型的目标维度
        
        all_coeffs = coeff.flatten()  # 将系数矩阵平铺成一维数组
        squared_energy = np.square(all_coeffs)  # 计算系数的能量（平方）
        sorted_indices = np.argsort(-squared_energy)  # 按能量降序排序系数的索引
        top_indices = sorted_indices[:target_dim]  # 选取能量最大的系数的索引
        
        reduced_coeff = np.zeros_like(all_coeffs)  # 创建全零数组，用于存放降维后的系数
        np.put(reduced_coeff, top_indices, np.take(all_coeffs, top_indices))  # 仅在选定的索引位置放置原系数
        
        reduced_coeffs.append(reduced_coeff.reshape(coeff.shape))  # 将降维后的系数重塑回原始形状，并添加到列表
    
    return reduced_coeffs

In [ ]:
def process_and_save_reduced_coeffs(input_dir, output_dir, target_dims):
    """
    加载npz文件，对系数进行降维，然后保存降维后的结果。
    
    参数:
    input_dir: 包含npz文件的输入目录。
    output_dir: 保存降维后的npz文件的输出目录。
    target_dims: 字典，指定每种系数的目标维度。
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # 如果输出目录不存在，则创建
    
    for filename in os.listdir(input_dir):  # 遍历输入目录中的所有文件
        if filename.lower().endswith('.npz'):  # 确保文件是npz格式
            npz_file_path = os.path.join(input_dir, filename)  # 构建完整的文件路径
            cA, cH, cV, cD = load_coeffs(npz_file_path)  # 加载系数
            reduced_coeffs = reduce_dimensions_separately([cA, cH, cV, cD], target_dims)  # 对系数进行降维
            
            # 构建输出文件的路径
            output_filename = os.path.splitext(filename)[0] + '_reduced.npz'
            output_file_path = os.path.join(output_dir, output_filename)
            
            # 保存降维后的系数到npz文件
            np.savez_compressed(output_file_path, cA=reduced_coeffs[0], cH=reduced_coeffs[1], cV=reduced_coeffs[2], cD=reduced_coeffs[3])

In [0]:
# 示例用法
input_dir = 'path_to_your_npz_files'  # 替换为你的小波系数npz文件所在的目录路径
output_dir = 'path_to_save_reduced_coeffs'  # 替换为你希望保存降维后系数的目录路径
target_dims = {'cA': 600, 'cH': 133, 'cV': 133, 'cD': 134}  # 设置每种类型系数的目标维度

process_and_save_reduced_coeffs(input_dir, output_dir, target_dims)  # 执行处理和保存操作
